In [18]:
import sys
import os
sys.path.insert(0, '../../Utilities/')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io

In [15]:
# Часть кода которая выполняет AC
class PhysicsInformedNN:
    def __init__(self, x0, u0, x1, layers, dt, lb, ub, q):
        self.x0 = x0
        self.u0 = u0
        self.x1 = x1
        self.layers = layers
        self.dt = dt
        self.lb = lb
        self.ub = ub
        self.q = q
        # Initialize neural networks
        self.net_U0 = self.build_net()
        self.net_U1 = self.build_net()
        # Initialize IRK weights (you need to define the logic for weights)
        self.IRK_weights = self.initialize_IRK_weights()
        # Initialize optimizer
        self.optimizer = tf.keras.optimizers.Adam()

    def build_net(self):
        """Build the neural network."""
        model = tf.keras.Sequential()
        for layer_size in self.layers:
            model.add(tf.keras.layers.Dense(layer_size, activation='tanh'))
        return model

    def initialize_IRK_weights(self):
        """Initialize IRK weights (This needs to be customized)."""
        return np.array([[1]])  # Example value

    def net_U1(self, x):
        return self.net_U1(x)  # Return the output of the neural network net_U1 for input x

    def train(self, nIter):
        for it in range(nIter):
            with tf.GradientTape() as tape:
                # Compute predictions for the initial condition x0 and the boundary condition x1
                U0_pred = self.net_U0(self.x0)
                U1_pred = self.net_U1(self.x1)
                
                # Calculate the loss
                loss = tf.reduce_sum(tf.square(self.u0 - U0_pred)) + \
                       tf.reduce_sum(tf.square(U1_pred[0, :] - U1_pred[1, :]))
            
            # Compute gradients of the loss with respect to the trainable variables of both networks
            grads = tape.gradient(loss, self.net_U0.trainable_variables + self.net_U1.trainable_variables)
            
            # Apply gradients to update the weights of the networks
            self.optimizer.apply_gradients(zip(grads, self.net_U0.trainable_variables + self.net_U1.trainable_variables))
    
            # Optional: Print the loss every few iterations
            if it % 10 == 0:
                print(f"Iteration {it}, Loss: {loss.numpy()}")
                
    def fwd_gradients_0(self, U, x):
        """Compute forward gradients of U with respect to x."""
        with tf.GradientTape() as tape:
            tape.watch(x)
            g = U
        grad = tape.gradient(g, x)
        if grad is None:
            raise ValueError("Gradient calculation failed, grad is None.")
        return grad

    def net_U0(self, x):
        """Compute the U0 prediction."""
        U1 = self.net_U1(x)
        U = U1[:, :-1]  # Remove the last dimension if it's not needed
        if U is None:
            raise ValueError("Output U is None, check the neural network output.")
        U_x = self.fwd_gradients_0(U, x)
        U_xx = self.fwd_gradients_0(U_x, x)
        F = 5.0 * U - 5.0 * U**3 + 0.0001 * U_xx
        U0 = U1 - self.dt * tf.matmul(F, self.IRK_weights.T)
        return U0

    def predict(self, x_star):
        """Predict using the trained model."""
        return self.net_U0(x_star)

In [16]:
# Скачивания жанных и прописания начальных паараметров

print(os.getcwd())

# Load the data from the .mat file
data = scipy.io.loadmat('AC.mat')

# Extract the necessary variables
t = data['tt'].flatten()[:, None]  # Time values
x = data['x'].flatten()[:, None]   # Spatial values
Exact = np.real(data['uu']).T      # Real part of 'uu', transposed for correct shape

# Select indices for initialization
idx_t0 = 20
idx_t1 = 180

iterations = 5 * 10 ** 4   # Примерно при 5000 - 10000 будет норм

# Initialize data
N = 100  # Number of spatial points to select
idx_x = np.random.choice(Exact.shape[1], N, replace=False)  # Randomly select N points

x0 = x[idx_x, :]  # Spatial points at t0
u0 = Exact[idx_t0:idx_t0+1, idx_x].T  # Corresponding initial condition data

# Boundary conditions
lb = x.min()  # Lower boundary (min value of x)
ub = x.max()  # Upper boundary (max value of x)


In [17]:
# Create the model and train it
model = PhysicsInformedNN(x0, u0, x1, layers, dt, lb, ub, q)
model.train(100)  # Training for 100 iterations (this is just a placeholder)

# Make predictions
x_star = x  # Assuming we want predictions over all x points
U1_pred = model.predict(x_star)  # Make predictions using the trained model# Create the model
model = PhysicsInformedNN(x0, u0, x1, layers, dt, lb, ub, q)

# Train the model
model.train(iterations)  # Training for 100 iterations

# Make predictions after training
x_star = x  # Assuming we want predictions over all x points
U1_pred = model.predict(x_star)  # Make predictions using the trained model


Текущий рабочий каталог: C:\Users\77003\JupyterLab\PINN_course\HW_2


AttributeError: 'PhysicsInformedNN' object has no attribute 'build_model'

In [ ]:
# Plotting the results
import matplotlib.gridspec as gridspec

# Define the grid for subplots
gs1 = gridspec.GridSpec(1, 2)

# First plot at time t0
ax1 = plt.subplot(gs1[0, 0])
ax1.plot(x, Exact[idx_t0, :], 'b-', linewidth=2, label='Exact')  # Exact solution at t0
ax1.plot(x0, u0, 'rx', linewidth=2, label='Data')  # Initial condition
ax1.set_xlabel('$x$')
ax1.set_ylabel('$u(t, x)$')
ax1.set_title('$t = %.2f$' % (t[idx_t0]), fontsize=10)
ax1.set_xlim([lb - 0.1, ub + 0.1])
ax1.legend(loc='upper center', bbox_to_anchor=(0.8, -0.3), ncol=2, frameon=False)

# Second plot at time t1
ax2 = plt.subplot(gs1[0, 1])
ax2.plot(x, Exact[idx_t1, :], 'b-', linewidth=2, label='Exact')  # Exact solution at t1
ax2.plot(x_star, U1_pred.numpy()[:, 0], 'r--', linewidth=2, label='Prediction')  # Predicted solution
ax2.set_xlabel('$x$')
ax2.set_ylabel('$u(t, x)$')
ax2.set_title('$t = %.2f$' % (t[idx_t1]), fontsize=10)
ax2.set_xlim([lb - 0.1, ub + 0.1])
ax2.legend(loc='upper center', bbox_to_anchor=(0.1, -0.3), ncol=2, frameon=False)

# Display the plots
plt.tight_layout()
plt.show()